In [ ]:
composer_folder = 'Dataset'
test_folder = 'Test'
output_folder = 'Output'

model_tag = 'Fundamental_MidiAI'

In [1]:
lr = 3e-4
n_epochs = 1000
seq_len = 128
mini_batch_size = 1024
batch_size = 1024

mini_batch_cycles = batch_size//mini_batch_size

# For max normalization, making sure that target can be a little bit longer than maximum sequence delta
stretch = 1.1

# GPUs
device_ids = [3,2,1,0]
device = device_ids[0]

batch_display_freq = 1000
save_audio_freq = 5000

insignificant_velocity= 15
insignificant_timestep = 8

In [2]:
import os

import mido
import numpy as np

import math

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset
import torch.nn.functional as f

import pandas as pd
from scipy import stats

import random

In [3]:
def encode_track(track, test=False):

    filtered_track = []
    
    full_delay = 0
    for i,msg in enumerate(track):
        full_delay += msg.time
        if msg.type in ['note_on', 'note_off']:
            is_on = (msg.type == 'note_on')
            note = msg.note
            velocity = msg.velocity

            filtered_track.append({'delay':full_delay, 
                                   'is_on':is_on, 
                                   'velocity':velocity,
                                   'note':note
                                   })
            full_delay = 0

    assert (filtered_track != []), "(!) Empty Track"
    
    encoded_track = []
    delay = 0
    for i,msg in enumerate(filtered_track):
        delay += msg['delay']
        if msg['is_on'] and  msg['velocity'] > 0:
            length = 0
            for j in range(i+1,len(filtered_track)):
                length += filtered_track[j]['delay']
                if filtered_track[j]['note'] == msg['note']: break
            
            if delay > 0: encoded_track.append([delay, 0, 128])                
            encoded_track.append([length, msg['velocity'], msg['note']])
            
            delay = 0
    
    df = pd.DataFrame(encoded_track, columns = ["Length", "Velocity", "Note"]) 
    
    #Remove smallest accidental delays
    df.drop(df[(df['Note'] == 128) & (df['Length'] < insignificant_timestep)].index, inplace=True)
    
    return df


def decode_track(encoded_track):

    expand_track = []
    
    track_time = 0
    for msg in encoded_track:
        
        length = max(0,msg[0])
        velocity = max(0,min(msg[1],127))
        note = max(0,min(msg[2],128))
        
        if note == 128 : track_time += length
        else:
            expand_track.append({
                'time':track_time, 
                'is_on':True, 
                'velocity':velocity,
                'note':note})

            expand_track.append({
                'time':track_time+length, 
                'is_on':False, 
                'velocity':velocity,
                'note':note
                })       
    
    expand_track.sort(key=lambda msg: msg['time'])
    
    track_out = mido.MidiTrack()
    
    previous_time = 0
    for msg in expand_track:
        time = int(msg['time'])
        delta =  time - previous_time
        velocity = int(msg['velocity'])
        note = int(msg['note'])        
        
        if msg['is_on']:
            track_out.append(mido.Message('note_on', 
                                           note=note, 
                                           velocity=velocity, 
                                           time=delta))
        elif not msg['is_on']:
            track_out.append(mido.Message('note_off', 
                                           note=note, 
                                           velocity=velocity, 
                                           time=delta))
        previous_time = time
        
    return track_out    

def read_midi(path):
    if os.path.isdir(path): return None
    mid_in = mido.MidiFile(path)
    return mid_in

def write_midi(mid_out, path):
    mid_out.save(path)

In [4]:
# Test
# load_path = os.path.join('.','midi_files',composer_folder)

# datasets = []
# file_name = os.listdir(load_path)[0]
# print(file_name)
# midi_path = os.path.join(load_path, file_name)
# mid_in = read_midi(midi_path)

# merged_tracks = mido.merge_tracks(mid_in.tracks)
# encoded_track = encode_track(merged_tracks)

# track = decode_track(encoded_track.values.tolist())

# mid_out = mido.MidiFile(type=0)
# mid_out.tracks.append(track)
# write_midi(mid_out, './{}.mid'.format(file_name))

In [5]:
def create_dataset(encoded_track, seq_len, test):
    
    encoded_track = encoded_track.to_numpy()
    features_list = []
    targets_list = []
    for i in range(len(encoded_track)-(seq_len+1)):
        features = encoded_track[i:i+seq_len+1,:]        
        features_list.append(features)
        if test: break
    feature_tensors = torch.FloatTensor(np.array(features_list))
    dataset = TensorDataset(feature_tensors)
    
    return dataset

In [6]:
def get_dataset(path, test=False):
    
    load_path = os.path.join('.','midi_files',path)

    datasets = []
    files = os.listdir(load_path)
    num_files = len(files)
    for i, file_name in enumerate(files):
        print("\rProgress: {:.2f}%".format(100 * i/num_files), end="")        
        
        try:
            if (file_name[-4:] == ".mid") or (file_name[-5:] == ".midi"):
                midi_path = os.path.join(load_path, file_name)
                mid_in = read_midi(midi_path)

                merged_tracks = mido.merge_tracks(mid_in.tracks)
                encoded_track = encode_track(merged_tracks)

                datasets.append(create_dataset(encoded_track, seq_len, test))
        except:
            print("\nIssue with file : {}".format(file_name))
            
    return ConcatDataset(datasets)

In [7]:
dataset = get_dataset(composer_folder)
test_dataset = get_dataset(test_folder, test=True)

data_loader = DataLoader(dataset, shuffle=True, batch_size=mini_batch_size)
test_data_loader = DataLoader(test_dataset, shuffle=False, batch_size=mini_batch_size)

# data_loader = torch.load('./dataloader.pth')
# test_data_loader = torch.load('./test_data_loader.pth')

In [8]:
class RelativeGlobalAttention(nn.Module):
    def __init__(self, d_model, num_heads, max_len=256, dropout=0.1):
        super().__init__()
        d_head, remainder = divmod(d_model, num_heads)
        if remainder:
            raise ValueError(
                "incompatible `d_model` and `num_heads`"
            )
        self.max_len = max_len
        self.d_model = d_model
        self.num_heads = num_heads
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.query = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.Er = nn.Parameter(torch.randn(max_len, d_head))
        self.register_buffer(
            "mask", 
            torch.tril(torch.ones(max_len, max_len))
            .unsqueeze(0).unsqueeze(0)
        )
        # self.mask.shape = (1, 1, max_len, max_len)

    
    def forward(self, x):
        # x.shape == (batch_size, seq_len, d_model)
        batch_size, seq_len, _ = x.shape
        
        if seq_len > self.max_len:
            raise ValueError(
                "sequence length exceeds model capacity"
            )
        
        k_t = self.key(x).reshape(batch_size, seq_len, self.num_heads, -1).permute(0, 2, 3, 1)
        # k_t.shape = (batch_size, num_heads, d_head, seq_len)
        v = self.value(x).reshape(batch_size, seq_len, self.num_heads, -1).transpose(1, 2)
        q = self.query(x).reshape(batch_size, seq_len, self.num_heads, -1).transpose(1, 2)
        # shape = (batch_size, num_heads, seq_len, d_head)
        
        start = self.max_len - seq_len
        Er_t = self.Er[start:, :].transpose(0, 1)
        # Er_t.shape = (d_head, seq_len)
        QEr = torch.matmul(q, Er_t)
        # QEr.shape = (batch_size, num_heads, seq_len, seq_len)
        Srel = self.skew(QEr)
        # Srel.shape = (batch_size, num_heads, seq_len, seq_len)
        
        QK_t = torch.matmul(q, k_t)
        # QK_t.shape = (batch_size, num_heads, seq_len, seq_len)
        attn = (QK_t + Srel) / math.sqrt(q.size(-1))
        mask = self.mask[:, :, :seq_len, :seq_len]
        # mask.shape = (1, 1, seq_len, seq_len)
        attn = attn.masked_fill(mask == 0, float("-inf"))
        # attn.shape = (batch_size, num_heads, seq_len, seq_len)
        attn = f.softmax(attn, dim=-1)
        out = torch.matmul(attn, v)
        # out.shape = (batch_size, num_heads, seq_len, d_head)
        out = out.transpose(1, 2)
        # out.shape == (batch_size, seq_len, num_heads, d_head)
        out = out.reshape(batch_size, seq_len, -1)
        # out.shape == (batch_size, seq_len, d_model)
        return self.dropout(out)
        
    
    def skew(self, QEr):
        # QEr.shape = (batch_size, num_heads, seq_len, seq_len)
        padded = f.pad(QEr, (1, 0))
        # padded.shape = (batch_size, num_heads, seq_len, 1 + seq_len)
        batch_size, num_heads, num_rows, num_cols = padded.shape
        reshaped = padded.reshape(batch_size, num_heads, num_cols, num_rows)
        # reshaped.size = (batch_size, num_heads, 1 + seq_len, seq_len)
        Srel = reshaped[:, :, 1:, :]
        # Srel.shape = (batch_size, num_heads, seq_len, seq_len)
        return Srel

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, max_len=256, dropout=0.2):
        super().__init__()
            
        self.heads = nn.ModuleList(
            [RelativeGlobalAttention(d_model, num_heads, max_len=128, dropout=0.2) for _ in range(num_heads)]
        )
        
        self.linear = nn.Linear(num_heads * d_model, d_model, bias=False)

    def forward(self, x):
        return self.linear(
            torch.cat([h(x) for h in self.heads], dim=-1)
        )
    
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, num_heads, max_len=256, dropout=0.2):
        super().__init__()
                 
        self.self_attention = MultiHeadAttention(d_model, num_heads, max_len=128, dropout=0.2)
        
        self.linear_layer = nn.Sequential(
            nn.Linear(d_model, d_model, bias=False),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model, bias=False),
        )
                 
        self.norm = nn.LayerNorm((seq_len,d_model))
        
    def forward(self, x):

        x = self.norm(self.self_attention(x))
        x = self.norm(self.linear_layer(x) + x)
                 
        return x   

In [10]:

# Relative position code taken from : https://github.com/evelinehong/Transformer_Relative_Position_PyTorch/blob/master/relative_position.py
# All credits goes to @evelinehong
class RelativePosition(nn.Module):
    
    def __init__(self, dim_hid, max_rel_pos):
        super().__init__()
        
        self.dim_hid = dim_hid
        self.max_rel_pos = max_rel_pos
        self.embeddings_table = nn.Parameter(torch.Tensor(max_rel_pos*2 + 1, self.dim_hid))
        nn.init.xavier_uniform_(self.embeddings_table)

    def forward(self, length_q, length_k):
        
        range_vec_q = torch.arange(length_q)
        range_vec_k = torch.arange(length_k)
        
        distance_mat = range_vec_k[None, :] - range_vec_q[:, None]
        distance_mat_clipped = torch.clamp(distance_mat, -self.max_rel_pos, self.max_rel_pos)
        
        final_mat = distance_mat_clipped + self.max_rel_pos
        final_mat = torch.LongTensor(final_mat).cuda()
        
        embeddings = self.embeddings_table[final_mat].cuda()

        return embeddings

class AttentionHead(nn.Module):
    def __init__(self, dim_in, dim_hid):
        super().__init__()
        
        self.dim_hid = dim_hid
        
        self.q = nn.Linear(dim_in, self.dim_hid, bias=False)
        self.k = nn.Linear(dim_in, self.dim_hid, bias=False)
        self.v = nn.Linear(dim_in, self.dim_hid, bias=False)    

    def forward(self, query, key, value, rel_pos_k, rel_pos_v, mask=False):

        rel_k = rel_pos_k(seq_len, seq_len)
        rel_v = rel_pos_v(seq_len, seq_len)
        
        q = self.q(query)
        k = self.k(key)
        v = self.v(value)

        if mask:
            mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).to(query.device)
            mask[mask.bool()] = -float('inf')
        else:
            mask = 0.

        # Classic attention
        cls_attn = q.bmm(k.transpose(1, 2))
        
        # Relative attention    
        rel_attn = q.permute(1,0,2).bmm(rel_k.permute(0,2,1)).permute(1,0,2)
        
        attn = (cls_attn + rel_attn) / cls_attn.size(-1) ** 0.5
        attn = f.softmax(attn + mask, dim=-1)
        
        x = attn.bmm(v)
        x += attn.permute(1,0,2).matmul(rel_v).permute(1,0,2)
        
        return x   

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, dim_in, dim_hid):
        super().__init__()
        
        self.max_rel_pos = seq_len   
        
        self.rel_pos_k = RelativePosition(dim_hid, self.max_rel_pos)
        self.rel_pos_v = RelativePosition(dim_hid, self.max_rel_pos) 
        
        self.heads = nn.ModuleList(
            [AttentionHead(dim_in, dim_hid) for _ in range(num_heads)]
        )
        self.linear = nn.Linear(num_heads * dim_hid, dim_in, bias=False)

    def forward(self, query, key, value, mask=0.):
        
        return self.linear(
            torch.cat([h(query, key, value, self.rel_pos_k, self.rel_pos_v, mask) for h in self.heads], dim=-1)
        )
    
class TransformerEncoder(nn.Module):
    def __init__(self, num_heads=32, dim_in=64, dim_hid=32, dim_ff=128, dropout=0.2):
        super().__init__()
                 
        self.dim_in = dim_in
        self.dim_ff = dim_ff
        self.dim_hid = dim_hid
        
        self.num_heads = num_heads
        self.dropout = dropout

        self.self_attention = MultiHeadAttention(num_heads=self.num_heads, dim_in=self.dim_in, dim_hid=self.dim_hid)
        
        self.linear_layer = nn.Sequential(
            nn.Linear(self.dim_in, self.dim_ff, bias=False),
            nn.GELU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.dim_ff, self.dim_in, bias=False),
        )
                 
        self.norm = nn.LayerNorm((seq_len,self.dim_in))
        
    def forward(self, x, mask=0.):

        x = self.norm(self.self_attention(x,x,x, mask) + x)
        x = self.norm(self.linear_layer(x) + x)
                 
        return x 



In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [12]:
class MidiAI(nn.Module):
    def __init__(self, num_layers=14, num_heads=12, embedding_size=64, dim_hid=32, dim_ff=128, dropout=0.2):
        super().__init__()

        self.num_layers = num_layers
        
        self.num_heads = num_heads
        self.dim_hid = dim_hid
        self.dim_ff = dim_ff
        
        self.embedding_size = embedding_size
        self.dropout = dropout
        
        self.embedding = nn.Embedding(129,self.embedding_size-3)
        
        layers = []
        for _ in range(self.num_layers):
            layer = TransformerEncoder(num_heads=self.num_heads, dim_in=self.embedding_size, dim_hid=self.dim_hid, dim_ff=self.dim_ff, dropout=self.dropout)
            layer = nn.DataParallel(layer, device_ids=device_ids)
            layers.append(layer)
        self.layers = nn.Sequential(*layers)
                
        self.linear = nn.Linear(self.embedding_size,131)
        self.norm = nn.LayerNorm((seq_len,self.embedding_size))
        
        self.embedding = nn.DataParallel(self.embedding, device_ids=device_ids)
        self.linear = nn.DataParallel(self.linear, device_ids=device_ids)
        self.norm = nn.DataParallel(self.norm, device_ids=device_ids)             
    
    def prepare_input(self, x_in):

        with torch.no_grad():
            x = x_in.clone()
            x[x[:,:,2] < 128] = 0.
            
            time_stamps = torch.cumsum(x[:,:,0], dim=1)
            max_value = time_stamps[:,-1:].clone()
            time_stamps /= max_value

        x = torch.zeros(x_in.shape[0], x_in.shape[1], self.embedding_size-1).to(x_in.device)
        x[:,:,0:3] = x_in.clone()
        x = torch.cat([time_stamps.unsqueeze(-1), x], dim=2)
        
        x[:,:,3:] = self.embedding(x[:,:,3].long()) * math.sqrt(self.embedding_size)
        
        return x
    
    def forward(self, x_in):

        x = self.prepare_input(x_in)

        for layer in self.layers:
             x = layer(x, mask=True)
        
        x = self.linear(x)

        return x

In [13]:
m = MidiAI().to(device)

In [14]:
print("Parameters = {:,.0f}".format(count_parameters(m)))

Parameters = 2,098,048


In [15]:
#m

In [16]:
optimizer = torch.optim.Adam(m.parameters(), lr=lr)
#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, epochs=n_epochs, steps_per_epoch=len(data_loader)//mini_batch_cycles)

In [17]:
bce_loss = nn.BCEWithLogitsLoss()
mse_loss = nn.MSELoss()
crossentropy_loss = nn.CrossEntropyLoss()

In [18]:
def extend_composition(model, x_in, extension_length, max_length):
    with torch.no_grad():
        context = None

        x_out = x_in.clone()
        for i in range(extension_length):
            y = model(x_out[:,i:i+x_in.shape[1],:])
            y[:,:,2] = y[:,:,2:].argmax(2)
            x_out = torch.cat([x_out,y[:,-1:,:3].clone()], dim=1)

        x_out[:,:,0] = x_out[:,:,0] * max_length
        x_out[:,:,1] = x_out[:,:,1] * 128

    return x_out[:,:,:3]

def swap_on_condition(t, fct, rdn_trsh = 0.5):
    t = t.numpy()
    n = t.shape[1]
    for k in range(t.shape[0]):
        for i in range(n-1):
            if fct(t[k,:,:],i) and (random.random()<rdn_trsh):
                t[k,i,:], t[k,i+1,:] = t[k,i+1,:], t[k,i,:].copy()
    return torch.tensor(t)

def add_delta(x, delta):
    
    x += torch.Tensor([delta]).to(x.device)

    floor = torch.FloatTensor([0]).to(x.device)
    ceil = torch.FloatTensor([127]).to(x.device)
    x = torch.max(x, floor.expand_as(x))
    x = torch.min(x, ceil.expand_as(x))
    
    return x

def prepare_batch(x, velocity_delta=0, note_delta=0, noise=False):
    with torch.no_grad():
        max_length = x[:,:,0].max(1)[0]
        max_length = torch.mul(max_length, stretch).reshape(-1,1)

        v_delta = random.randint(-velocity_delta, velocity_delta) 
        n_delta = random.randint(-note_delta, note_delta)

        if noise:
            x[:,:,0] += torch.randint(-insignificant_timestep, insignificant_timestep, size=x[:,:,0].shape)  #length noise
            x[:,:,1] += torch.randint(-insignificant_velocity, insignificant_velocity, size=x[:,:,1].shape)  #velocity noise

        x[:,:,1] = add_delta(x[:,:,1], v_delta) #velocity delta
        x[:,:,2][x[:,:,2]<128] = add_delta(x[:,:,2][x[:,:,2]<128], n_delta) #note delta

        x[:,:,0] /= max_length #length
        x[:,:,1] /= 127 #velocity

        #Randomly swap notes if happening at the sime time (not a 'wait')
        x = swap_on_condition(x, lambda x,i : ((x[i,2] != 128) and (x[i+1,2] != 128) and (i+1<=x.shape[0])), rdn_trsh=0.5)

    return x, max_length

In [ ]:
for i_epoch in range(1, n_epochs+1):
    
    velocity_losses = []
    length_losses = []
    note_losses = []
    
    optimizer.zero_grad()
    
    for i_batch, data in enumerate(data_loader,1):
            
        m.train()
        seq = data[0]
        
        seq,max_length = prepare_batch(seq, velocity_delta=20, note_delta=12, noise=True)
        
        x = seq[:,:-1,:].to(device)
        target = seq[:,1:,:].to(device)

        y = m(x)

        y_length = y[:,:,0]
        y_velocity = y[:,:,1]
        y_note = y[:,:,2:].permute(0,2,1)
        
        target_length = target[:,:,0]
        target_velocity = target[:,:,1]
        target_note = target[:,:,2]
        
        length_loss = mse_loss(y_length, target_length)
        velocity_loss = mse_loss(y_velocity, target_velocity)
        note_loss = crossentropy_loss(y_note, target_note.long())
        
        loss = length_loss + velocity_loss + note_loss
        
        velocity_losses.append(velocity_loss.cpu().detach())
        length_losses.append(length_loss.cpu().detach())
        note_losses.append(note_loss.cpu().detach())
        
        loss = length_loss + velocity_loss + note_loss
        loss.backward()
        
        if i_batch % mini_batch_cycles == 0:

            torch.nn.utils.clip_grad_norm_(m.parameters(), max_norm=8, norm_type=2.0)
            optimizer.step()
            #scheduler.step()
            optimizer.zero_grad()
        
        with torch.no_grad():
            if i_batch % batch_display_freq == 0:
                print("{} | {} | Losses | velocity = {:.2e} | length = {:.2e} | note = {:.2e} | all = {:.2e}".format(
                    i_epoch,
                    i_batch,
                    np.array(velocity_losses).mean(),
                    np.array(length_losses).mean(),
                    np.array(note_losses).mean(),
                    np.array(length_losses+note_losses+velocity_losses).mean()
                ))    

                with open('{}_log.txt'.format(model_tag), 'a') as log:
                    log.write("{} | {} | lr = {:.2e} | velocity = {:.2e} | length = {:.2e} | note = {:.2e} | all = {:.2e}\n".format(
                    i_epoch,
                    i_batch,
                    lr,
                    np.array(velocity_losses).mean(),
                    np.array(length_losses).mean(),
                    np.array(note_losses).mean(),
                    np.array(length_losses+note_losses+velocity_losses).mean()
                ))

                velocity_losses = []
                length_losses = []
                note_losses = []

            if i_batch % save_audio_freq == 0:      

                write_folder = os.path.join('.',output_folder,'{}_e{}_b{}'.format(model_tag,i_epoch,i_batch))
                if not os.path.exists(write_folder): os.makedirs(write_folder) 

                data = next(iter(test_data_loader))
                seq = data[0]

                # Normalize
                seq,max_length = prepare_batch(seq)
                x = seq[:,:-1,:].to(device)
                m = m.to(device)
                max_length = max_length.to(device)

                x_ext = extend_composition(m, x, extension_length=seq_len, max_length=max_length)

                for i in range(x_ext.shape[0]):
                    track = decode_track(x_ext[i,:,:].float().detach().cpu().numpy())
                    mid_out = mido.MidiFile(type=0)
                    mid_out.tracks.append(track)

                    file_path = os.path.join(write_folder, './{}_TestOutput_e{}_b{}_{}.mid'.format(model_tag,i_epoch,i_batch,i)) 

                    write_midi(mid_out, file_path)

                write_folder = os.path.join('.',output_folder,'{}_e{}_b{}'.format(model_tag,i_epoch,i_batch,i))
                if not os.path.exists(write_folder): os.makedirs(write_folder)

                file_path = os.path.join(write_folder,'{}_model_e{}_b{}_{}.pt'.format(model_tag, i_epoch, i_batch, i))

                torch.save(m.state_dict(), file_path)

1 | 1000 | Losses | velocity = 6.31e-02 | length = 3.88e-02 | note = 2.72e+00 | all = 9.40e-01
1 | 2000 | Losses | velocity = 4.93e-02 | length = 3.48e-02 | note = 2.26e+00 | all = 7.80e-01
1 | 3000 | Losses | velocity = 4.45e-02 | length = 3.42e-02 | note = 2.07e+00 | all = 7.16e-01
1 | 4000 | Losses | velocity = 4.30e-02 | length = 3.38e-02 | note = 1.95e+00 | all = 6.76e-01
1 | 5000 | Losses | velocity = 4.11e-02 | length = 3.36e-02 | note = 1.84e+00 | all = 6.40e-01
1 | 6000 | Losses | velocity = 3.94e-02 | length = 3.34e-02 | note = 1.74e+00 | all = 6.04e-01
1 | 7000 | Losses | velocity = 3.85e-02 | length = 3.31e-02 | note = 1.65e+00 | all = 5.74e-01
1 | 8000 | Losses | velocity = 3.76e-02 | length = 3.29e-02 | note = 1.57e+00 | all = 5.48e-01
1 | 9000 | Losses | velocity = 3.68e-02 | length = 3.26e-02 | note = 1.51e+00 | all = 5.28e-01
1 | 10000 | Losses | velocity = 3.56e-02 | length = 3.20e-02 | note = 1.45e+00 | all = 5.07e-01
1 | 11000 | Losses | velocity = 3.43e-02 | length